In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
!kaggle datasets download -d atharv1610/terrain-recognition

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/terrain-recognition.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
zip_ref

In [ ]:
import pathlib
data_dir=pathlib.Path("/content/Data Main/test")

In [ ]:
image_count = len(list(data_dir.glob('*/*.png')))
print(image_count)


In [ ]:
import PIL

In [ ]:
M = list(data_dir.glob('Marshy/*'))
PIL.Image.open(str(M[110]))

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.9,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
#AUTOTUNE = tf.data.AUTOTUNE

#train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
#val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
import tensorflow as tf

num_classes = 5

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128),
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=20
)

In [ ]:
model.summary()

In [ ]:
validation_accurarcies = []
model_names = ['CNN', 'VGG16', 'ResNet50', 'MobileNetV2', 'EfficientNetB0']

In [ ]:
# Evaluate the model on training and validation datasets
train_loss, train_accuracy = model.evaluate(train_ds, verbose=0)
val_loss, val_accuracy = model.evaluate(val_ds, verbose=0)
validation_accurarcies.append(val_accuracy)
# Print the evaluation metrics
print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")


In [ ]:
from sklearn.metrics import classification_report
import numpy as np
true_labels = []
predicted_labels = []
for images, labels in val_ds:
    predictions = model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)
    true_labels.extend(labels.numpy())
    predicted_labels.extend(predicted_classes)
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)
report = classification_report(true_labels, predicted_labels, target_names=val_ds.class_names, digits=4)
print(report)


In [ ]:
import matplotlib.pyplot as plt

# Extract metrics from history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

# Plot Accuracy
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot Loss
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2, InceptionV3, EfficientNetB0
from tensorflow.keras import layers, models, optimizers

# Function to create a transfer learning model
def create_transfer_model(base_model, num_classes, input_shape=(180, 180, 3)):
    base_model.trainable = False  # Freeze the pre-trained layers
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),  # Pooling layer
        layers.Dense(128, activation='relu'),  # Fully connected layer
        layers.Dropout(0.5),  # Regularization
        layers.Dense(num_classes, activation='softmax')  # Output layer
    ])
    return model


VGG16

In [ ]:
# Import TensorFlow one-hot encoding function
import tensorflow as tf

# Get the number of classes (5 in this case, based on your dataset)
num_classes = len(train_ds.class_names)

# Function to convert integer labels to one-hot encoded labels
def one_hot_encode(image, label):
    label = tf.one_hot(label, depth=num_classes)
    return image, label

# Apply one-hot encoding to both training and validation datasets
train_ds = train_ds.map(one_hot_encode, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(one_hot_encode, num_parallel_calls=tf.data.AUTOTUNE)

# Prefetch the datasets to optimize performance
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)


In [ ]:
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(180, 180, 3))
vgg16_model = create_transfer_model(vgg16_base, num_classes)

vgg16_model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                    loss='categorical_crossentropy', metrics=['accuracy'])

print("Training VGG16 Model")
history1 = vgg16_model.fit(train_ds, validation_data=val_ds, epochs=5)


In [ ]:
vgg16_model.summary()

In [ ]:
# Evaluate the model on training and validation datasets
train_loss, train_accuracy = vgg16_model.evaluate(train_ds, verbose=0)
val_loss, val_accuracy = vgg16_model.evaluate(val_ds, verbose=0)
validation_accurarcies.append(val_accuracy)
# Print the evaluation metrics
print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")


In [ ]:
from sklearn.metrics import classification_report
import numpy as np
true_labels = []
predicted_labels = []
for images, labels in val_ds:
    predictions = vgg16_model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)
    true_labels.extend(labels.numpy())
    predicted_labels.extend(predicted_classes)
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)
report = classification_report(true_labels, predicted_labels, target_names=class_names, digits=4)
print(report)


In [ ]:
import matplotlib.pyplot as plt

# Extract metrics from history
acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']
loss = history1.history['loss']
val_loss = history1.history['val_loss']
epochs_range = range(len(acc))

# Plot Accuracy
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot Loss
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

ResNet50

In [ ]:
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(180, 180, 3))
resnet_model = create_transfer_model(resnet_base, num_classes)

resnet_model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                     loss='categorical_crossentropy', metrics=['accuracy'])

print("Training ResNet50 Model")
history2 = resnet_model.fit(train_ds, validation_data=val_ds, epochs=5)


In [ ]:
# Evaluate the model on training and validation datasets
train_loss, train_accuracy = resnet_model.evaluate(train_ds, verbose=0)
val_loss, val_accuracy = resnet_model.evaluate(val_ds, verbose=0)
validation_accurarcies.append(val_accuracy)
# Print the evaluation metrics
print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")


In [ ]:
from sklearn.metrics import classification_report
import numpy as np
true_labels = []
predicted_labels = []
for images, labels in val_ds:
    predictions = resnet_model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)
    true_labels.extend(labels.numpy())
    predicted_labels.extend(predicted_classes)
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)
report = classification_report(true_labels, predicted_labels, target_names=class_names, digits=4)
print(report)


In [ ]:
resnet_model.summary()

In [ ]:
import matplotlib.pyplot as plt

# Extract metrics from history
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']
loss = history2.history['loss']
val_loss = history2.history['val_loss']
epochs_range = range(len(acc))

# Plot Accuracy
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot Loss
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

MobileNetV2

In [ ]:
mobilenet_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(180, 180, 3))
mobilenet_model = create_transfer_model(mobilenet_base, num_classes)

mobilenet_model.compile(optimizer=optimizers.Adam(learning_rate=0.001),

                        loss='categorical_crossentropy', metrics=['accuracy'])

print("Training MobileNetV2 Model")
history3 = mobilenet_model.fit(train_ds, validation_data=val_ds, epochs=5)


In [ ]:
mobilenet_model.summary()

In [ ]:
# Evaluate the model on training and validation datasets
train_loss, train_accuracy = mobilenet_model.evaluate(train_ds, verbose=0)
val_loss, val_accuracy = mobilenet_model.evaluate(val_ds, verbose=0)
validation_accurarcies.append(val_accuracy)
# Print the ev
print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")


In [ ]:
from sklearn.metrics import classification_report
import numpy as np
true_labels = []
predicted_labels = []
for images, labels in val_ds:
    predictions = mobilenet_model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)
    true_labels.extend(labels.numpy())
    predicted_labels.extend(predicted_classes)
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)
report = classification_report(true_labels, predicted_labels, target_names=class_names, digits=4)
print(report)


In [ ]:
import matplotlib.pyplot as plt

# Extract metrics from history
acc = history3.history['accuracy']
val_acc = history3.history['val_accuracy']
loss = history3.history['loss']
val_loss = history3.history['val_loss']
epochs_range = range(len(acc))

# Plot Accuracy
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot Loss
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

EfficientNetB0

In [ ]:
efficientnet_base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(180, 180, 3))
efficientnet_model = create_transfer_model(efficientnet_base, num_classes)

efficientnet_model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                           loss='categorical_crossentropy', metrics=['accuracy'])

print("Training EfficientNetB0 Model")
history5 = efficientnet_model.fit(train_ds, validation_data=val_ds, epochs=5)


In [ ]:
efficientnet_model.summary()

In [ ]:
# Evaluate the model on training and validation datasets
train_loss, train_accuracy = efficientnet_model.evaluate(train_ds, verbose=0)
val_loss, val_accuracy = efficientnet_model.evaluate(val_ds, verbose=0)
validation_accurarcies.append(val_accuracy)
# Print the evaluation metrics
print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")


In [ ]:
from sklearn.metrics import classification_report
import numpy as np
true_labels = []
predicted_labels = []
for images, labels in val_ds:
    predictions = efficientnet_model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)
    true_labels.extend(labels.numpy())
    predicted_labels.extend(predicted_classes)
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)
report = classification_report(true_labels, predicted_labels, target_names=val_ds.class_names, digits=4)
print(report)


In [ ]:
import matplotlib.pyplot as plt

# Extract metrics from history
acc = history5.history['accuracy']
val_acc = history5.history['val_accuracy']
loss = history5.history['loss']
val_loss = history5.history['val_loss']
epochs_range = range(len(acc))

# Plot Accuracy
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot Loss
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
plt.bar(range(len(model_names)), validation_accurarcies, color='skyblue') # Use index as x-axis
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Accuracies of Different Models on the Same Dataset')
plt.xticks(range(len(model_names)), model_names, rotation=45) # Set xtick labels explicitly
plt.ylim(0, 1)
plt.show()